<a href="https://colab.research.google.com/github/meghananav/Commercial_Insights_Forecasting/blob/main/Commercial_Insights_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Commercial Insights Forecasting Project

This project demonstrates how **time series forecasting** and **machine learning** can be applied to generate actionable commercial insights.  
Using simulated sales data across products and regions, the notebook walks through an end-to-end workflow of data preprocessing, feature engineering, visualization, and forecasting.

## Objectives
- **Simulate a commercial sales dataset** with product, region, pricing, promotion, and customer segment details  
- **Perform exploratory data analysis (EDA)** to identify patterns and correlations  
- **Engineer time series and promotional features** to enrich the dataset  
- **Train and evaluate forecasting models** using **XGBoost** (machine learning) and **Prophet** (time series)  
- **Visualize and interpret forecasts** to support data-driven business decision-making


## 1. Import Required Libraries


In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn xgboost prophet plotly

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from prophet import Prophet
from datetime import datetime, timedelta

np.random.seed(42)


## 2. Simulate Commercial Sales Data


In [ ]:
num_days = 365
products = ['Product_A', 'Product_B', 'Product_C', 'Product_D']
regions = ['North', 'South', 'East', 'West']

dates = [datetime.today() - timedelta(days=x) for x in range(num_days)]
data = []

for date in dates:
    for product in products:
        for region in regions:
            sales_quantity = np.random.randint(20, 200)
            price = np.random.uniform(10, 100)
            promotion = np.random.choice([0, 1], p=[0.8, 0.2])
            customer_segment = np.random.choice(['Retail', 'Wholesale'])
            data.append([date, product, region, sales_quantity, round(price,2), promotion, customer_segment])

df = pd.DataFrame(data, columns=['Date','Product_ID','Region','Sales_Quantity','Price','Promotion','Customer_Segment'])
df.head()


## 3. Exploratory Data Analysis (EDA)


In [ ]:
# 1. Data overview
print("Data types:")
print(df.dtypes)
print("\nMissing values:")
print(df.isnull().sum())
df.describe()


In [ ]:
# 2. Sales trend over time
sales_time = df.groupby('Date')['Sales_Quantity'].sum()
plt.figure(figsize=(12,6))
plt.plot(sales_time.index, sales_time.values)
plt.title("Total Sales Over Time")
plt.xlabel("Date")
plt.ylabel("Sales Quantity")
plt.grid(True)
plt.show()



In [ ]:
# 3. Total sales by product
plt.figure(figsize=(8,5))
sns.barplot(data=df, x='Product_ID', y='Sales_Quantity', estimator=sum)
plt.title("Total Sales by Product")
plt.ylabel("Total Sales")
plt.show()


In [ ]:
# 4. Total sales by region
region_sales = df.groupby('Region')['Sales_Quantity'].sum()
region_sales.plot(kind='bar', figsize=(6,4), title="Total Sales by Region")
plt.ylabel("Total Sales")
plt.show()


In [ ]:
# 5. Correlation heatmap
numeric_cols = ['Sales_Quantity', 'Price', 'Promotion']
plt.figure(figsize=(6,4))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()


In [ ]:
# 6. Distribution of sales quantity
plt.figure(figsize=(8,5))
sns.histplot(df['Sales_Quantity'], bins=30, kde=True)
plt.title("Distribution of Sales Quantity")
plt.xlabel("Sales Quantity")
plt.show()


In [ ]:
# 7. Impact of promotions
plt.figure(figsize=(8,5))
sns.boxplot(data=df, x='Promotion', y='Sales_Quantity')
plt.title("Impact of Promotion on Sales Quantity")
plt.xlabel("Promotion (0=No, 1=Yes)")
plt.ylabel("Sales Quantity")
plt.show()


## 4. Feature Engineering

In this section, we enhance the dataset with additional features to improve model performance.  
Steps include:
- Extracting temporal features (day, month, weekday)  
- Encoding categorical variables (product, region, customer segment)  
- Creating interaction features (promotion impact)  
- Generating lag features (previous day and previous week sales)  


In [ ]:
# Convert date column to datetime (if not already)
df['Date'] = pd.to_datetime(df['Date'])

# Extract temporal features
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Weekday'] = df['Date'].dt.weekday  # 0=Monday, 6=Sunday


In [ ]:
# Encode categorical features
df_encoded = df.copy()
df_encoded['Product_ID'] = df_encoded['Product_ID'].astype('category').cat.codes
df_encoded['Region'] = df_encoded['Region'].astype('category').cat.codes
df_encoded['Customer_Segment'] = df_encoded['Customer_Segment'].astype('category').cat.codes


In [ ]:
# Create interaction feature: Promotion impact
df_encoded['Sales_Promo_Impact'] = df_encoded['Sales_Quantity'] * df_encoded['Promotion']


In [ ]:
# Create lag features for time series context
df_encoded['Prev_Day_Sales'] = df_encoded.groupby(['Product_ID','Region'])['Sales_Quantity'].shift(1)
df_encoded['Prev_Week_Sales'] = df_encoded.groupby(['Product_ID','Region'])['Sales_Quantity'].shift(7)

# Fill missing lag values with 0
df_encoded.fillna(0, inplace=True)


In [ ]:
# Show first few rows of engineered dataset
df_encoded.head()


## 5. Forecasting with XGBoost

In this section, we apply **XGBoost Regression** to forecast sales quantities.  
Steps include:
- Defining features and target  
- Splitting the data into training and testing sets  
- Training an XGBoost model  
- Evaluating model performance using RMSE  


In [ ]:
# Define features and target
features = [
    'Product_ID', 'Region', 'Price', 'Promotion', 'Customer_Segment',
    'Day', 'Month', 'Weekday', 'Sales_Promo_Impact', 'Prev_Day_Sales', 'Prev_Week_Sales'
]
target = 'Sales_Quantity'

X = df_encoded[features]
y = df_encoded[target]


In [ ]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
# Initialize and train XGBoost model
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)
model.fit(X_train, y_train)


In [ ]:
# Make predictions
y_pred = model.predict(X_test)


In [ ]:
# Evaluate performance
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"XGBoost Model RMSE: {rmse:.2f}")


## 6. Forecasting with Prophet

In this section, we apply **Facebook Prophet** for time series forecasting.  
Steps include:
- Aggregating daily sales  
- Training the Prophet model  
- Forecasting the next 30 days  
- Visualizing forecast results  


In [ ]:
# Prepare dataset for Prophet: aggregate daily sales
df_prophet = df.groupby('Date')['Sales_Quantity'].sum().reset_index()
df_prophet = df_prophet.rename(columns={'Date': 'ds', 'Sales_Quantity': 'y'})
df_prophet.head()


In [ ]:
# Initialize and fit Prophet model
m = Prophet(daily_seasonality=True)
m.fit(df_prophet)


In [ ]:
# Create future dataframe (30 days ahead)
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)


In [ ]:
# Plot forecast
fig1 = m.plot(forecast)
plt.title("Sales Forecast for Next 30 Days (Prophet)")
plt.xlabel("Date")
plt.ylabel("Sales Quantity")
plt.show()


In [ ]:
# Plot forecast components (trend, seasonality)
fig2 = m.plot_components(forecast)
plt.show()


## 7. Model Evaluation and Comparison

In this section, we compare the performance of **XGBoost** and **Prophet**.  
We evaluate forecast accuracy using **Root Mean Squared Error (RMSE)** and visualize predicted vs. actual sales.  


In [ ]:
# --- Evaluate Prophet ---
# Merge Prophet forecast with actuals
df_compare = df_prophet.merge(forecast[['ds','yhat']], on='ds', how='left')

# Only compare where actual values exist
mask = df_compare['y'].notnull()
y_true_prophet = df_compare.loc[mask, 'y']
y_pred_prophet = df_compare.loc[mask, 'yhat']

mse_prophet = mean_squared_error(y_true_prophet, y_pred_prophet)
rmse_prophet = mse_prophet ** 0.5
print(f"Prophet Model RMSE: {rmse_prophet:.2f}")


In [ ]:
# --- Already have XGBoost RMSE ---
print(f"XGBoost Model RMSE: {rmse_tuned:.2f}" if 'rmse_tuned' in globals() else f"XGBoost Model RMSE: {rmse:.2f}")


In [ ]:
# --- Side-by-side comparison ---
results = pd.DataFrame({
    'Model': ['XGBoost', 'Prophet'],
    'RMSE': [rmse_tuned if 'rmse_tuned' in globals() else rmse, rmse_prophet]
})
results


In [ ]:
# --- Visualization: Actual vs Prophet Forecast ---
plt.figure(figsize=(12,6))
plt.plot(df_compare['ds'], df_compare['y'], label="Actual Sales")
plt.plot(df_compare['ds'], df_compare['yhat'], label="Prophet Forecast")
plt.title("Prophet: Actual vs Forecasted Sales")
plt.xlabel("Date")
plt.ylabel("Sales Quantity")
plt.legend()
plt.show()


In [ ]:
# --- Visualization: Actual vs XGBoost Predictions (Test set only) ---
plt.figure(figsize=(8,5))
plt.scatter(y_test, y_pred if 'y_pred' in globals() else y_pred_tuned, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red')
plt.title("XGBoost: Actual vs Predicted Sales")
plt.xlabel("Actual Sales")
plt.ylabel("Predicted Sales")
plt.show()


## 8. Business Insights

In this section, we extract key insights from the simulated sales dataset.  
These insights highlight **which products, regions, and promotions drive sales performance**,  
providing value for commercial decision-making.


In [ ]:
# Top products by total sales
top_products = df.groupby('Product_ID')['Sales_Quantity'].sum().sort_values(ascending=False)
print("Total Sales by Product:")
print(top_products)
top_products.plot(kind='bar', figsize=(6,4), title="Total Sales by Product")
plt.ylabel("Sales Quantity")
plt.show()


In [ ]:
# Top regions by total sales
region_sales = df.groupby('Region')['Sales_Quantity'].sum().sort_values(ascending=False)
print("Total Sales by Region:")
print(region_sales)
region_sales.plot(kind='bar', figsize=(6,4), title="Total Sales by Region", color="teal")
plt.ylabel("Sales Quantity")
plt.show()


In [ ]:
# Promotion impact on average sales
promo_sales = df.groupby('Promotion')['Sales_Quantity'].mean()
print("Average Sales With/Without Promotion:")
print(promo_sales)
promo_sales.plot(kind='bar', figsize=(6,4), title="Impact of Promotion on Sales", color=["grey","green"])
plt.ylabel("Average Sales Quantity")
plt.xticks([0,1], ["No Promotion","Promotion"], rotation=0)
plt.show()


In [ ]:
# Customer segment analysis
segment_sales = df.groupby('Customer_Segment')['Sales_Quantity'].sum()
print("Total Sales by Customer Segment:")
print(segment_sales)
segment_sales.plot(kind='bar', figsize=(6,4), title="Total Sales by Customer Segment", color="orange")
plt.ylabel("Sales Quantity")
plt.show()


## 9. Visualization of Forecasts

In this section, we visualize the forecasts generated by **Prophet** and **XGBoost**.  
This helps compare predicted sales against actual values and highlights how well each model captures sales patterns.


In [ ]:
# Prophet: Actual vs Forecasted Sales
plt.figure(figsize=(12,6))
plt.plot(df_compare['ds'], df_compare['y'], label="Actual Sales")
plt.plot(df_compare['ds'], df_compare['yhat'], label="Prophet Forecast")
plt.title("Prophet: Actual vs Forecasted Sales")
plt.xlabel("Date")
plt.ylabel("Sales Quantity")
plt.legend()
plt.show()


In [ ]:
# Prophet forecast components (trend & seasonality)
fig2 = m.plot_components(forecast)
plt.show()


In [ ]:
# XGBoost: Actual vs Predicted Sales (Test set)
plt.figure(figsize=(8,5))
plt.scatter(y_test, y_pred_tuned if 'y_pred_tuned' in globals() else y_pred, alpha=0.5, label="Predictions")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', label="Perfect Prediction Line")
plt.title("XGBoost: Actual vs Predicted Sales")
plt.xlabel("Actual Sales")
plt.ylabel("Predicted Sales")
plt.legend()
plt.show()


In [ ]:
# Side-by-side comparison table for Prophet vs XGBoost
results = pd.DataFrame({
    'Model': ['XGBoost', 'Prophet'],
    'RMSE': [rmse_tuned if 'rmse_tuned' in globals() else rmse, rmse_prophet]
})
print("Model Comparison (Lower RMSE = Better Performance):")
results


## 10. Limitations & Improvements

### Limitations
- Dataset is **synthetic**, so results may differ with real-world sales data.  
- Models are trained on a **limited set of features** (product, region, promotion, etc.).  
- Prophet may struggle with sudden spikes, while XGBoost depends heavily on feature engineering.  

### Improvements
- Include **external factors** like holidays, campaigns, or economic trends.  
- Explore **hybrid models** combining Prophet (trend/seasonality) with XGBoost (complex patterns).  
- Deploy the best-performing model as an **API or dashboard** for real-time insights.  


## 11. Conclusion & Key Insights

This project showcased a complete **forecasting pipeline** using a simulated commercial sales dataset.  
We applied both **XGBoost** and **Prophet** models to predict future sales and compared their performance.  

### Key Takeaways
- **XGBoost** performed well in capturing complex, non-linear relationships.  
- **Prophet** provided interpretable forecasts with clear trend and seasonality insights.  
- Business analysis showed that **promotions significantly increased sales**, and certain **products and regions consistently outperformed others**.  

### Business Value
The forecasting framework can support:  
- **Inventory planning** by predicting demand  
- **Promotion strategy** by quantifying promotional impact  
- **Regional targeting** for better resource allocation  

